In [16]:
import os

In [17]:
%pwd

'c:\\Users\\User\\Desktop\\PROJECTS\\animal classifier\\Animal-classifier'

In [18]:
os.chdir("../")

In [19]:
%pwd

'c:\\Users\\User\\Desktop\\PROJECTS\\animal classifier'

In [20]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list


In [21]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf

In [22]:
class ConfigurationManager:

    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        #prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "catsndogs")
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config

In [23]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
import time

In [24]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config
    
 
    def train_valid_generator(self):

        train_datagen_kwargs = dict(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True
        )

        training_set_kwargs = dict(
                                #'dataset/training_set',
                                target_size = (64, 64),
                                batch_size=self.config.params_batch_size,
                                class_mode = 'binary'
        )
        test_datagen_kwargs = dict(rescale = 1./255)

        test_set_kwargs = dict(#'dataset/test_set',
                            target_size = (64, 64),
                            batch_size=self.config.params_batch_size,
                            class_mode = 'binary')


        # dataflow_kwargs = dict(
        #     target_size=self.config.params_image_size[:-1],
        #     batch_size=self.config.params_batch_size,
        #     interpolation="bilinear"
        #)

        # valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
        #     **datagenerator_kwargs
        # )

        test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
            **test_datagen_kwargs
        )

        self.test_set = test_datagen.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **test_set_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
                **train_datagen_kwargs
            )
        else:
            train_datagen = test_datagen

        self.training_set = train_datagen.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **training_set_kwargs
        )

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)


    # def train(self, callback_list: list):
    #     self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
    #     self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

    #     self.model.fit(
    #         self.train_generator,
    #         epochs=self.config.params_epochs,
    #         steps_per_epoch=self.steps_per_epoch,
    #         validation_steps=self.validation_steps,
    #         validation_data=self.valid_generator,
    #         callbacks=callback_list
        #)

        # self.save_model(
        #     path=self.config.trained_model_path,
        #     model=self.model
        #)
    def build_cnn_model(self):
        self.cnn = cnn
        cnn = tf.keras.models.Sequential()
        cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=self.config.params_image_size))
        cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
        cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
        cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
        cnn.add(tf.keras.layers.Flatten())
        cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))
        cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))
        cnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

        return self.cnn
    
        

    def train_cnn_model(self):
         self.model.fit(x=self.training_set, validation_data=self.test_set, epochs=self.config.params_epochs)

         self.save_model(
                    path=self.config.trained_model_path,
                    model=self.model)

        # # Usage
        # cnn_model = build_cnn_model()
        # train_cnn_model(cnn_model, training_set, test_set, epochs=2)



In [25]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    

except Exception as e:
    raise e

AttributeError: 'ConfigurationManager' object has no attribute 'config'